In [ ]:
"""
main.py - Main script for training Noise2Void with NAFNet
"""

import os
import torch
from torch.utils.data import DataLoader

from N2V_Vanilla.config import create_config, TrainingConfig
from N2V_Vanilla.models import create_nafnet
from N2V_Vanilla.losses import create_loss_function
from N2V_Vanilla.dataset import NoisyPatchDataset, ValidationDataset
from N2V_Vanilla.trainer import create_trainer
from N2V_Vanilla.utils import setup_reproducibility, print_model_info, create_directories

# ---------------------------
# === Main Training Function
# ---------------------------
def main(config: TrainingConfig):
    # Validate configuration
    config.validate_config()
    
    # Setup reproducibility
    setup_reproducibility(config.seed)
    
    # Get device
    device = torch.device(config.get_device())
    print(f"Using device: {device}")
    
    # Create output directories
    create_directories(config.output_dir)
    
    # Create datasets
    print("\nCreating datasets...")
    train_ds = NoisyPatchDataset(
        root_dir=config.train_dir,
        patch_size=config.patch_size,
        dataset_size=config.dataset_size
    )
    
    val_ds = ValidationDataset(
        gt_dir=config.val_gt_dir,
        noisy_dir=config.val_noisy_dir,
        normalize=False  # Normalization handled in utils
    ) if config.val_gt_dir and config.val_noisy_dir else None
    
    # Create dataloaders
    train_loader = DataLoader(
        train_ds,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        pin_memory=config.pin_memory
    )
    
    val_loader = DataLoader(
        val_ds,
        batch_size=1,  # Process one GT pair at a time
        shuffle=False,
        num_workers=0,
        pin_memory=config.pin_memory
    ) if val_ds else None
    
    # Create model
    print("\nCreating NAFNet model...")
    model = create_nafnet(config.nafnet_config).to(device)
    print_model_info(model, config.nafnet_config)
    
    # Create loss function
    print(f"\nCreating {config.loss_type} loss function...")
    criterion = create_loss_function(
        loss_type=config.loss_type,
        device=device,
        **config.loss_weights
    )
    
    # Create optimizer
    optimizer_params = {
        'lr': config.learning_rate,
        'weight_decay': config.weight_decay,
        'betas': config.betas
    }
    if config.optimizer_type.lower() == "adam":
        optimizer = torch.optim.Adam(model.parameters(), **optimizer_params)
    elif config.optimizer_type.lower() == "adamw":
        optimizer = torch.optim.AdamW(model.parameters(), **optimizer_params)
    else:
        raise ValueError(f"Unknown optimizer type: {config.optimizer_type}")
    
    # Create scheduler
    scheduler = None
    if config.scheduler_type.lower() == "step":
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=config.step_size,
            gamma=config.gamma
        )
    elif config.scheduler_type.lower() == "cosine":
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=int(config.epochs/5)
        )
    elif config.scheduler_type.lower() == "plateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='max',
            factor=0.5,
            patience=5,
            verbose=True
        )
    else:
        print(f"Warning: Unknown scheduler type '{config.scheduler_type}', no scheduler will be used.")
    
    # Create trainer
    trainer_config = config.to_dict()
    trainer = create_trainer(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        config=trainer_config
    )
    
    # Optional: Resume from checkpoint
    # resume_path = "./training_output/checkpoints/checkpoint_epoch_XX.pth"
    # if os.path.exists(resume_path):
    #     trainer.resume_training(resume_path)
    
    # Start training
    trainer.train(
        train_loader=train_loader,
        val_loader=val_loader,
        epochs=config.epochs
    )

if __name__ == "__main__":
    # Create configuration
    # Options: "default", "fast", "high_quality", "cpu"
    config = create_config("default")
    
    # Optional: Update paths if needed
    config.update_paths(
        train_dir=r"E:\PHD\phd_env\Proyectos\Denoising_challenge\Calcium\Data\Train\slices",
        val_gt_dir=r"E:\PHD\phd_env\Proyectos\Denoising_challenge\Calcium\Data\Val\GT\slices",
        val_noisy_dir=r"E:\PHD\phd_env\Proyectos\Denoising_challenge\Calcium\Data\Val\noisy\slices",
        output_dir="./training_output"
    )
    
    # Optional: Customize further
    config.epochs = 50
    config.batch_size = 32
    config.learning_rate = 5e-3
    config.model_size = 'medium'
    config.update_model_config(model_size='medium')
    config.loss_type = "advanced"
    config.loss_weights['ssim_weight'] = 0.3
    config.num_workers = 8
    config.scheduler_type.lower() == "step"
    
    main(config)

Using device: cuda

Creating datasets...
Found 6000 training images in E:\PHD\phd_env\Proyectos\Denoising_challenge\Calcium\Data\Train\slices
Found 1500 validation pairs

Creating NAFNet model...


C:\Users\Guill\miniconda3\envs\phd_env_conda\Lib\site-packages\torchmetrics\utilities\prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


MODEL INFORMATION
Architecture: NAFNet
Parameters: 1,128,161
Configuration:
  img_channel: 1
  width: 32
  middle_blk_num: 1
  enc_blk_nums: [1, 1, 1]
  dec_blk_nums: [1, 1, 1]

Creating advanced loss function...
Starting training for 50 epochs...

Epoch 1/50
----------------------------------------


Epoch 1 [Train]: 100%|████████████████████████████████████████████████| 625/625 [11:10<00:00,  1.07s/it, loss=0.034142]


Train Loss: 0.034142
Learning Rate: 5.00e-03


...

Epoch 31 [Train]: 100%|███████████████████████████████████████████████| 625/625 [11:09<00:00,  1.07s/it, loss=0.033885]


Train Loss: 0.033885
Learning Rate: 1.25e-03


Epoch 31 [Val]: 100%|█████████████████████████████████████████| 20/20 [00:31<00:00,  1.59s/it, psnr=33.007, ssim=0.750]


Val PSNR: 34.9752, Val SSIM: 0.8004

Epoch 32/50
----------------------------------------


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000028126484360>  1.09s/it, loss=0.033948]
Traceback (most recent call last):
  File "C:\Users\Guill\miniconda3\envs\phd_env_conda\Lib\site-packages\torch\utils\data\dataloader.py", line 1650, in __del__
    self._shutdown_workers()
  File "C:\Users\Guill\miniconda3\envs\phd_env_conda\Lib\site-packages\torch\utils\data\dataloader.py", line 1614, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\Guill\miniconda3\envs\phd_env_conda\Lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Guill\miniconda3\envs\phd_env_conda\Lib\multiprocessing\popen_spawn_win32.py", line 112, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 
Epoch 32 [Train]:  74%|██████████████████████████████████▋      